In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/Users/elyhahami/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
import os
import cv2

dataset_path = "/Users/elyhahami/Downloads/Columbia Gaze Data Set"
file_list = sorted(os.listdir(dataset_path))
file_list.pop(0)
file_list

['0001',
 '0002',
 '0003',
 '0004',
 '0005',
 '0006',
 '0007',
 '0008',
 '0009',
 '0010',
 '0011',
 '0012',
 '0013',
 '0014',
 '0015',
 '0016',
 '0017',
 '0018',
 '0019',
 '0020',
 '0021',
 '0022',
 '0023',
 '0024',
 '0025',
 '0026',
 '0027',
 '0028',
 '0029',
 '0030',
 '0031',
 '0032',
 '0033',
 '0034',
 '0035',
 '0036',
 '0037',
 '0038',
 '0039',
 '0040',
 '0041',
 '0042',
 '0043',
 '0044',
 '0045',
 '0046',
 '0047',
 '0048',
 '0049',
 '0050',
 '0051',
 '0052',
 '0053',
 '0054',
 '0055',
 '0056']

In [ ]:
#can optimize this code by: 1) improve variable names; use numpy to deal with flattening arrays
##also need to get training labels
#can also do data augmentation
import numpy as np
import os
import cv2
my_list = []
dataset_path = "/Users/elyhahami/Downloads/Columbia Gaze Data Set"

# Get a list of subject folders in the dataset
file_list = sorted(os.listdir(dataset_path))
file_list.pop(0)
file_list
# Iterate over each subject folder
for subject_folder in file_list:
    subject_folder_path = os.path.join(dataset_path, subject_folder)

    # Get a list of photo files in the subject folder
    photo_files = sorted(os.listdir(subject_folder_path))
    my_list.append(photo_files)

# print(my_list)
labels_list = []
empty_list = []
bad_list = ['Thumbs.db','.picasa.ini', '.DS_Store']
for i in range(len(my_list)):
    photo_name_list = my_list[i]
    new_list = [item for item in photo_name_list if item not in bad_list]
    for j in new_list:
        label = j.split('_')[2][:-1]
        labels_list.append(label)
    empty_list.append(new_list)
#finish something like this
# print(empty_list)

one_dim_empty_list = [element for sublist in empty_list for element in sublist]
# print(len(one_dim_empty_list))
# # print(one_dim_empty_list)
# print(labels_list)
# print(len(labels_list))
models_train_labels = np.array([int(x) for x in labels_list])
print(models_train_labels)
print(len(models_train_labels))
print(len(one_dim_empty_list))

[-15 -15 -15 ...  30  30  30]
5880
5880


In [ ]:
def process_images(starting, ending):
    processed_images = []

    for image_path in one_dim_empty_list[starting:ending]:
        full_image_path = dataset_path + '/' + image_path
        lolol = full_image_path.split('/')
        number_path = lolol[5]
        important_num = number_path[0:4]

        newest_full_image_path = dataset_path + '/' + str(important_num) + '/' + image_path
        image = cv2.imread(newest_full_image_path)
        resized_image = cv2.resize(image, (224, 224))# Resize the image to the desired dimensions (prop to div by 10)
        normalized_image = resized_image / 255.0 # Normalize the pixel values between 0 and 1

        processed_images.append(normalized_image)


    return processed_images

In [ ]:
def concatenate_all_images():
    batch_size = 100  # Adjust batch size according to your requirements
    starting_index = 0
    all_images = []

    while starting_index < len(one_dim_empty_list):
        processed_batch = process_images(starting_index, starting_index + batch_size)
        all_images.extend(processed_batch)
        starting_index += batch_size

    return all_images

In [ ]:
total_train_images = concatenate_all_images()

In [ ]:
#writing test case to see correspondence between models_train_labels and total_train_images:

In [ ]:
##one-hot encoding
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
label_encoder = LabelEncoder()
integer_labels = label_encoder.fit_transform(models_train_labels)
one_hot_labels = to_categorical(integer_labels, num_classes=5)

In [ ]:
total_train_images = np.array(total_train_images)

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16

# Set the number of classes
num_classes = 5

# Set the input shape for the model
input_shape = (224, 224, 3)  # Adjust based on your image dimensions

# Load pre-trained VGG16 model without the top (fully connected) layers
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Add your own classification layers on top of the VGG16 model
x = base_model_vgg16.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Add a Dropout layer with a dropout rate of 0.5

predictions = Dense(num_classes, activation='softmax')(x)

# Create the VGG16-based model
model = Model(inputs=base_model_vgg16.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model_vgg16.layers[:-5]:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

In [ ]:
history = model.fit(x=total_train_images, y=one_hot_labels,
          validation_split=0.2, batch_size = 128, epochs = 20, shuffle = True, verbose = 1,
                   callbacks=[reduce_lr]) # callbacks=[stop_early]

Epoch 1/20
37/37 [==============================] - 689s 19s/step - loss: 0.0634 - accuracy: 0.9838 - val_loss: 24.8542 - val_accuracy: 0.2764 - lr: 0.0010
Epoch 2/20
37/37 [==============================] - 672s 18s/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 17.1265 - val_accuracy: 0.2534 - lr: 0.0010
Epoch 3/20
37/37 [==============================] - 667s 18s/step - loss: 8.2144e-04 - accuracy: 1.0000 - val_loss: 11.8317 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 4/20
37/37 [==============================] - 666s 18s/step - loss: 3.1210e-04 - accuracy: 1.0000 - val_loss: 6.4994 - val_accuracy: 0.4065 - lr: 0.0010
Epoch 5/20
37/37 [==============================] - 666s 18s/step - loss: 2.1932e-04 - accuracy: 1.0000 - val_loss: 2.9500 - val_accuracy: 0.5553 - lr: 0.0010
Epoch 6/20
37/37 [==============================] - 667s 18s/step - loss: 1.6141e-04 - accuracy: 1.0000 - val_loss: 1.7073 - val_accuracy: 0.6794 - lr: 0.0010
Epoch 7/20
37/37 [==============================] -

In [ ]:
model.save("most_updated_head_position_prediction.h5")

In [ ]:
# model.save("pre-trained-VGG16-gaze-detection")

INFO:tensorflow:Assets written to: pre-trained-VGG16-gaze-detection/assets


INFO:tensorflow:Assets written to: pre-trained-VGG16-gaze-detection/assets


In [ ]:
# def head_direction_predictions(path):
#     import cv2
#     import numpy as np

#     # Load and preprocess the image
#     image_path = path
#     image = cv2.imread(image_path)
#     resized_image = cv2.resize(image, (224, 224))  # Resize the image to match the input size of your model
#     normalized_image = resized_image / 255.0  # Normalize the pixel values between 0 and 1
#     processed_image = np.expand_dims(normalized_image, axis=0)  # Add an extra dimension for batch size
#     predictions = model.predict(processed_image)
#     class_labels = ["-30 degrees", "-15 degrees", "0 degrees", "15 degrees", "30 degrees"]
#     predicted_class_index = np.argmax(predictions)
#     predicted_class_label = class_labels[predicted_class_index]
#     return "Predicted head position:", predicted_class_label

In [ ]:
# head_direction_predictions("/Users/elyhahami/Downloads/Columbia Gaze Data Set/0003/0003_2m_0P_0V_-5H.jpg")

1/1 [==============================] - 0s 136ms/step


('Predicted head position:', '0 degrees')